In [ ]:
corpus = ['king is a strong man', 
          'queen is a  beautiful woman']

In [2]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [3]:
def remove_stop_words(corpus):
    stop_words = ['is', 'a']
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
    
    return results

In [4]:
corpus = remove_stop_words(corpus)

In [5]:
words = []
for text in corpus:
    for word in text.split(' '):
        words.append(word)

words = set(words)
words

{'', 'beautiful', 'king', 'man', 'queen', 'strong', 'woman'}

In [6]:
word2int = {}

for i,word in enumerate(words):
    word2int[word] = i

sentences = []

for sentence in corpus:
    sentences.append(sentence.split())
    
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1] : 
            if neighbor != word:
                data.append([word, neighbor])

In [7]:
import pandas as pd
for text in corpus:
    print(text)

df = pd.DataFrame(data, columns = ['input', 'label'])

df.head(10)

king strong man
queen  beautiful woman


,input,label
0,king,strong
1,king,man
2,strong,king
3,strong,man
4,man,king
5,man,strong
6,queen,beautiful
7,queen,woman
8,beautiful,queen
9,beautiful,woman


In [8]:
word2int

{'': 0,
 'man': 1,
 'king': 2,
 'queen': 3,
 'beautiful': 4,
 'woman': 5,
 'strong': 6}

# Define Tensorflow Graph

In [9]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

ONE_HOT_DIM = len(words)

# function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))

# convert them to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)

# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# word embedding will be 2 dimension for 2d visualization
EMBEDDING_DIM = 2 

# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
# !pip install tensorflow==1.15

  Using cached tensorflow-1.15.0-cp37-cp37m-win_amd64.whl (295.1 MB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
Processing c:\users\user\appdata\local\pip\cache\wheels\21\7f\02\420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3\gast-0.2.2-py3-none-any.whl
  Using cached tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Attempting uninstall: gast
    Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Succ

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\user\\AppData\\Local\\Temp\\pip-uninstall-aqdpz4ye\\python\\_dtypes.pyd'
Consider using the `--user` option or check the permissions.



In [11]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 30000
for i in range(iteration):
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 3000 == 0:
        print('iteration '+str(i)+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

iteration 0 loss is :  3.3033535
iteration 3000 loss is :  0.9197665
iteration 6000 loss is :  0.89435
iteration 9000 loss is :  0.8805318
iteration 12000 loss is :  0.87182474
iteration 15000 loss is :  0.86573285
iteration 18000 loss is :  0.86117697
iteration 21000 loss is :  0.8576093
iteration 24000 loss is :  0.854719
iteration 27000 loss is :  0.8523161


In [12]:
# Now the hidden layer (W1 + b1) is actually the word look up table
vectors = sess.run(W1 + b1)
print(vectors)

[[-0.2277059   0.47798306]
 [-7.1597724  -1.5422564 ]
 [-0.78266484 -2.9098434 ]
 [ 7.354738   -0.5724599 ]
 [ 1.1047513   2.6811812 ]
 [ 0.6717469  -0.19374679]
 [-0.669384   -0.0171535 ]]


In [13]:
w2v_df = pd.DataFrame(vectors, columns = ['x1', 'x2'])
w2v_df['word'] = words
w2v_df = w2v_df[['word', 'x1', 'x2']]
w2v_df

,word,x1,x2
0,,-0.227706,0.477983
1,man,-7.159772,-1.542256
2,king,-0.782665,-2.909843
3,queen,7.354738,-0.572460
4,beautiful,1.104751,2.681181
5,woman,0.671747,-0.193747
6,strong,-0.669384,-0.017154


In [15]:
import re
from lxml import etree
import urllib.request
import zipfile
from nltk.tokenize import word_tokenize, sent_tokenize

with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    target_text = etree.parse(z.open('ted_en-20160408.xml', 'r'))
    parse_text = '\n'.join(target_text.xpath('//content/text()'))
    
parse_text[:500]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.\nConsider Facit. I'm actually old enough to remember them. Facit was a fantastic company. They were born deep in the Swedish forest, and they made the best mechanical calculators in the world. Everybody used them. A"

-------------------------------------------------------------------------------
-------------------------------------------------------------------------------

content_text = re.sub(r'\([^)]*\)', '', parse_text)
- 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
- 해당 코드는 괄호로 구성된 내용을 제거.

sent_text=sent_tokenize(content_text)
- 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
- 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
result = [word_tokenize(sentence) for sentence in normalized_text]

- 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
print('총 샘플의 개수 : {}'.format(len(result)))


In [24]:
content_text = re.sub(r'\([^)]*\)', '', parse_text)

sent_text = sent_tokenize(content_text)

normalized_text = []
for string in sent_text:
    tokens = re.sub(r'\([^a-z0-9]+', '', string.lower())
    normalized_text.append(tokens)
    
    
result = [word_tokenize(sentence) for sentence in normalized_text]

print('총 샘플의 개수 : {}'.format(len(result)))

for line in result[:3]:
    print(line)


총 샘플의 개수 : 273424
['here', 'are', 'two', 'reasons', 'companies', 'fail', ':', 'they', 'only', 'do', 'more', 'of', 'the', 'same', ',', 'or', 'they', 'only', 'do', 'what', "'s", 'new', '.']
['to', 'me', 'the', 'real', ',', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', ':', 'exploration', 'and', 'exploitation', '.']
['both', 'are', 'necessary', ',', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing', '.']


In [26]:
!pip install gensim
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

model_result = model.wv.most_similar('man')
print(model_result)

  Created wheel for smart-open: filename=smart_open-2.1.0-py3-none-any.whl size=110324 sha256=61c2812271a816dddfd12616fd714599b2678144c306a2c208d31eeafaf66205
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\56\b5\6d\86dbe4f29d4688e5163a8b8c6b740494310040286fca4dc648
Successfully built smart-open
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.15
    Uninstalling Cython-0.29.15:
      Successfully uninstalled Cython-0.29.15
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16
[('woman', 0.8600116968154907), ('lady', 0.7854600548744202), ('guy', 0.7826949954032898), ('boy', 0.7658523321151733), ('girl', 0.7556694149971008), ('gentleman', 0.7262185215950012), ('soldier', 0.7183092832565308), ('kid', 0.6886534690856934), ('poet', 0.678715705871582), ('david', 0.6573482155799866)]


In [29]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v')
loaded_model = KeyedVectors.load_word2vec_format('eng_w2v')

model_result = loaded_model.most_similar('man')
print(model_result)

[('woman', 0.8600116968154907), ('lady', 0.7854600548744202), ('guy', 0.7826949954032898), ('boy', 0.7658523321151733), ('girl', 0.7556694149971008), ('gentleman', 0.7262185215950012), ('soldier', 0.7183092832565308), ('kid', 0.6886534690856934), ('poet', 0.678715705871582), ('david', 0.6573482155799866)]


In [30]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename='ratings.txt')

train_data = pd.read_table('ratings.txt')

train_data[:5]

len(train_data)

train_data.isnull().values.sum()

train_data=train_data.dropna(how='any')

train_data.isnull().values.sum()

len(train_data)

# 정규식 표현을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace('[^ㄱ-하-ㅣ가-힣]','')



In [32]:
from konlpy.tag import Okt

# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

okt = Okt()
tokenized_data = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True)
    print('토큰화=>', temp_X)
    temp_X = [word for word in temp_X if not word in stopwords]
    
    #불용어 제거
    tokenized_data.append(temp_X)
    print('불용어 =>', temp_X)
    

from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, 
                workers = 4, sg = 0)

print(model.wv.most_similar('최민식'))
print(model.wv.most_similar('히어로'))

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.